In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import collections
import json
import math
import random

import IPython.display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn.metrics

import pmer

In [ ]:
dataset = pmer.datasets.Dota2Dataset.from_csv('dota2.csv')

In [ ]:
# Represent games between players as an undirected graph.

node_template = {
    'n_games': 0,
}

edge_template = {
    'n_games': 0,
}

nodes = collections.defaultdict(lambda: node_template.copy())
edges = collections.defaultdict(lambda: edge_template.copy())

for event in list(dataset)[-5000:]:
    player_ids = event.winners + event.losers
    for player1_id in player_ids:
        nodes[player1_id]['n_games'] += 1
    for player1_id in event.winners:
        for player2_id in event.losers:
            key = tuple(sorted([player1_id, player2_id]))
            edges[key]['n_games'] += 1

In [ ]:
# Convert nodes and edges to sigma.js format.

for i, (player_id, node) in enumerate(nodes.items()):
    node['id'] = str(player_id)
    node['label'] = str(player_id)
    node['x'] = 100 * math.cos(2 * i * math.pi / len(nodes))
    node['y'] = 100 * math.sin(2 * i * math.pi / len(nodes))
    node['size'] = math.log(node['n_games'] + 1)
    
for i, ((player1_id, player2_id), edge) in enumerate(edges.items()):
    # edge['id'] = '{}vs{}'.format(player1_id, player2_id)
    edge['id'] = str(i)
    edge['source'] = str(player1_id)
    edge['target'] = str(player2_id)

#graph = {
#    'nodes': list(nodes.values()),
#    'edges': list(edges.values()),
#}

#filename = 'graph.json'
#file = open(filename, 'w')
#json.dump(graph, file)
#file.flush()
    
# Bind the constructed graph to a global JS variable.
#IPython.display.Javascript('window.graph_filename = {};'.format(json.dumps(filename)))

In [ ]:
import csv

with open('graph.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Source', 'Target', 'Type', 'Weight'])
    for edge in edges.values():
        row = [edge['source'], edge['target'], 'undirected', float(edge['n_games'])]
        writer.writerow(row)

In [ ]:
import csv
import scipy.sparse

node_ids = sorted(nodes)

adj_matrix = scipy.sparse.lil_matrix((len(node_ids), len(node_ids)), dtype=np.float)

for i, source_id in enumerate(node_ids):
    for j, target_id in enumerate(node_ids[i+1:], i+1):
        edge = edges.get((source_id, target_id), None)
        if edge:
            adj_matrix[i, j] = edge['n_games']
            adj_matrix[j, i] = edge['n_games']

with open('graph.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow([''] + node_ids)
    for i, node_id in enumerate(node_ids):
        row = [node_id] + list(adj_matrix[i].toarray()[0])
        writer.writerow(row)

In [ ]:
%%HTML
<script src="jslib/jquery-2.1.4.min.js"></script>
<script src="jslib/sigma/sigma.min.js"></script>
<script src="jslib/sigma/plugins/sigma.parsers.json.min.js"></script>
<script src="jslib/sigma/plugins/sigma.layout.forceAtlas2.min.js"></script>
<style>
#sigma-container {
    max-width: 400px;
    height: 400px;
    margin: auto;
}
</style>
<div id="sigma-container"></div>

In [ ]:
%%javascript
sigma.parsers.json(window.graph_filename, {
    container: 'sigma-container',
}, function (s) {
    //debugger;
    s.startForceAtlas2({worker: true, barnesHutOptimize: false});
})